This notebook does the following:
*   Preprocess the corpus- lowercase, tokenize, bigram
*  Count the number of occurences of a set of pre-defined words in each text


Qualitative words:

**qualitative**

interview

ethnograph*


**Simulation/Model:**

simulation 

formal modal:


**Experiment (these would be collapsed into quantitative but it would be interesting to know this separately as well):**

field experiment

lab experiment 

Restart Kernel

In [ ]:
import nltk
import numpy as np
import pandas as pd
from nltk import word_tokenize
import re
import seaborn as sns
import matplotlib.pyplot as plt
import smart_open
#from gensim.models import LdaModel
#from gensim.models import CoherenceModel
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
#pyLDAvis.enable_notebook()
import spacy
import numpy as np
#from gensim.models import LdaModel



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Journal_Analysis/"

In [ ]:
def extract_documents(folder_name):
  import os
  docs = []
  allfiles = []
  for entry in os.scandir(folder_name):
      if entry.path.endswith(".txt") and entry.is_file():
          path_paper = entry.path
          allfiles.append(path_paper)
          if os.stat(path_paper).st_size == 0:
              continue
          with open(path_paper) as f:
            text = f.read()
            docs.append(text)
  return docs, allfiles

In [ ]:
docs, allfiles = extract_documents(path + 'allpapers_NoEssays')

In [ ]:
range(len(docs))

range(0, 220)

In [ ]:
toy = "I did an experiment."
toy.split().count("experiment")

0

**Get the majority method based on sum of number of times each keyword appears in the document**

In [ ]:
#Edits: Add "Experiment" in experiment
# Replace Hypothesis with quantitative in quantitative, add "statistically significant" and "statistical significance" 
# Mixed Methods 
# Edits Aug 4: 
# Add "independent variable", "control variable", "dependent variable" in quantitative
# Lower bar than 10 maybe 5
# Sum of counts in one category > average of all individual non-zero counts
# Just the word "qualitative" using split, same for "interview"
# Experiments in mixed 
# Split all the words to fix the stemming issue
# Make a stata file with all the things coded
# Pie charts for topics : showing break up of all other characteristics that we were looking at 
# Potential words for quantitative: "categorical variables", "logit", " standard error", "OLS", "linear model", "sensitivity analysis", "coefficients"
# Potential words for quantitative: "survey"
qualitative = ["interview" , "interviews", "ethnograph", "qualitative", "grounded theory"] 
simulation = ["simulation", "formal model"]
#experiment = ["field experiment", "lab experiment"]
experiment = ["field experiment", "lab experiment", "experiment", "experiments"]
#quantitative = ["regression", "hypothesis"]
quantitative = ["regression", "quantitative", "statistically significant", "statistical significance", "independent variable", "control variable", "dependent variable", "logit", "standard error", "OLS", "linear model"]
from collections import defaultdict
countsum = {}
for i in range(len(docs)):

  q = sum([docs[i].lower().split().count(x) for x in qualitative])
  s = sum([docs[i].lower().count(x) for x in simulation])
  e = sum([docs[i].lower().split().count(x) for x in experiment])
  qt = sum([docs[i].lower().count(x) for x in quantitative])
  countsum[allfiles[i].split("/")[-1].strip('.txt')] = [q,s,e,qt]


In [ ]:
df_countsum = pd.DataFrame(data = countsum, index = None).T

In [ ]:
df_countsum.columns = ["Qualitative", "Simulation", "Experiment", "Quantitative"]

In [ ]:
df_countsum_no_nulls = df_countsum[(df_countsum != 0).any(axis='columns')]

In [ ]:
#df_countsum_no_nulls["majority"] = df_countsum_no_nulls.idxmax(axis = 1)

**Change it to mixed category in case of a tie**

In [ ]:
df_countsum_no_nulls["majority"] = df_countsum_no_nulls.apply(lambda x: x.values == x.values.max(), axis = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_countsum_no_nulls["Majority_Method_Index"] = df_countsum_no_nulls["majority"].apply(lambda row: [i for i, x in enumerate(row) if x] )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
categories = df_countsum.columns
df_countsum_no_nulls["Majority_Method"] = df_countsum_no_nulls["Majority_Method_Index"].apply(lambda x : [categories[i] for i in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
categories = df_countsum.columns
categories = list(categories)
categories.append("Majority_Method")
categories

['Qualitative', 'Simulation', 'Experiment', 'Quantitative', 'Majority_Method']

In [ ]:
df_methods = df_countsum_no_nulls[categories]

In [ ]:
del df_countsum_no_nulls
#del df_countsum
#del df_ties
#del df_ties_morethan1

In [ ]:
df_methods.reset_index(inplace = True)


In [ ]:
df_methods.rename(columns = {"index" : "DOI"}, inplace = True)

In [ ]:
df_methods.head(10)

,DOI,Qualitative,Simulation,Experiment,Quantitative,Majority_Method
0,10.1177_0001839213486759,1,0,0,23,[Quantitative]
1,10.1177_0001839218793128,2,1,0,14,[Quantitative]
2,10.1177_0001839212437519,12,0,3,2,[Qualitative]
3,10.1177_0001839216673823,0,0,0,24,[Quantitative]
4,10.1177_0001839213488773,1,0,0,15,[Quantitative]
5,10.1177_0001839218761369,0,2,0,25,[Quantitative]
6,10.1177_0001839215619198,8,0,0,26,[Quantitative]
7,10.1177_0001839219855329,3,0,0,1,[Qualitative]
8,10.1177_0001839212461439,0,0,0,18,[Quantitative]
9,10.1177_0001839212440972,0,0,50,4,[Experiment]


In [ ]:
len(df_methods)

218

In [ ]:
df_methods[df_methods["DOI"] == "10.1177_0001839218771550"]

,DOI,Qualitative,Simulation,Experiment,Quantitative,Majority_Method
10,10.1177_0001839218771550,4,0,0,43,[Quantitative]


In [ ]:
df_methods["Majority_Method"].value_counts()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[Quantitative]                 137
[Qualitative]                   63
[Experiment]                     9
[Simulation]                     6
[Qualitative, Quantitative]      1
[Qualitative, Simulation]        1
[Experiment, Quantitative]       1
Name: Majority_Method, dtype: int64

**Get the word count of each keyword describing the methods**

In [ ]:
qualitative = ["interview" , "interviews", "ethnograph", "qualitative", "grounded theory"]
simulation = ["simulation", "formal model"]
#experiment = ["field experiment", "lab experiment"]
experiment = ["field experiment", "lab experiment", "experiment", "experiments"]
#quantitative = ["regression", "hypothesis"]
quantitative = ["regression", "quantitative", "statistically significant", "statistical significance", "independent variable", "control variable", "dependent variable", "logit", "standard error", "OLS", "linear model"]
from collections import defaultdict
countwords = {}
for i in range(len(docs)):

  q = [docs[i].lower().split().count(x) for x in qualitative]
  s = [docs[i].lower().count(x) for x in simulation]
  e = [docs[i].lower().split().count(x) for x in experiment]
  qt = [docs[i].lower().count(x) for x in quantitative]
  countwords[allfiles[i].split("/")[-1].strip('.txt')] = [q,s,e,qt]

In [ ]:
df_countwords = pd.DataFrame(data = countwords, index = None).T

In [ ]:
df_countwords.reset_index(inplace = True)

In [ ]:
df_countwords.columns = ["DOI", "[interview , interviews, ethnograph, qualitative, grounded theory]", "[simulation, formal model]", "[field experiment, lab experiment, experiment, experiments]", "[regression, quantitative, statistically significant, statistical significance, independent variable, control variable, dependent variable, logit, standard error, OLS, linear model]" ]

In [ ]:
df_countwords.to_excel(path + "methods.xlsx")

In [ ]:
df_countwords

,DOI,"[interview , interviews, ethnograph, qualitative, grounded theory]","[simulation, formal model]","[field experiment, lab experiment, experiment, experiments]","[regression, quantitative, statistically significant, statistical significance, independent variable, control variable, dependent variable, logit, standard error, OLS, linear model]"
0,10.1177_0001839213486759,"[0, 0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[7, 0, 1, 0, 2, 3, 3, 1, 6, 0, 0]"
1,10.1177_0001839218793128,"[0, 2, 0, 0, 0]","[1, 0]","[0, 0, 0, 0]","[4, 0, 2, 0, 1, 2, 3, 0, 2, 0, 0]"
2,10.1177_0001839212437519,"[4, 8, 0, 0, 0]","[0, 0]","[0, 0, 1, 2]","[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,10.1177_0001839216673823,"[0, 0, 0, 0, 0]","[0, 0]","[0, 0, 0, 0]","[6, 0, 0, 0, 3, 5, 7, 0, 3, 0, 0]"
4,10.1177_0001839213488773,"[0, 0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[4, 0, 0, 0, 0, 2, 3, 0, 5, 0, 1]"
...,...,...,...,...,...
215,10.1177_0001839220962795,"[4, 7, 0, 4, 0]","[0, 0]","[0, 0, 0, 0]","[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
216,asqu.2011.56.1.001,"[0, 0, 0, 0, 0]","[0, 0]","[0, 0, 30, 10]","[4, 1, 0, 0, 3, 0, 7, 0, 1, 0, 0]"
217,asqu.2011.56.1.061,"[0, 1, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[8, 1, 0, 0, 1, 3, 7, 0, 2, 0, 0]"
218,asqu.2011.56.1.026,"[0, 0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[4, 0, 8, 0, 0, 2, 3, 0, 0, 0, 0]"


**Mixed Category - any two of [Qualitative, Experiment, Quantitative]**





Decision Rule: Above average OR Above a number (10)

In [ ]:
df_merged = pd.merge(df_methods[["DOI", "Majority_Method"]], df_countwords, on = "DOI", how = "outer")

In [ ]:
df_merged["Majority_Method_cs"] = df_merged["Majority_Method"].dropna().apply(lambda x: ",".join(x))

In [ ]:
m = ["Qualitative", "Quantitative", "Experiment"]
df_merged_q = df_merged[(df_merged["Majority_Method_cs"] == "Qualitative") | (df_merged["Majority_Method_cs"] == "Quantitative") | (df_merged["Majority_Method_cs"] == "Experiment")].drop("Majority_Method", axis =1)
df_merged_q.columns = ["DOI", 'Qualitative', 'Simulation', 'Experiment', 'Quantitative', 'Majority_Method' ]

In [ ]:
df_merged_others = df_merged[(df_merged["Majority_Method_cs"] != "Qualitative") & (df_merged["Majority_Method_cs"] != "Quantitative") & (df_merged["Majority_Method_cs"] != "Experiment")].drop("Majority_Method", axis =1)
df_merged_others.columns = ["DOI", 'Qualitative', 'Simulation', 'Experiment', 'Quantitative', 'Majority_Method' ]

In [ ]:
import statistics
a = [0,2,3]. = 5
b = [3,4,1]. = 7
savg = 6
sum(a) > savg 
statistics.mean(a + b)
np.array(a + b)[np.nonzero(np.array(a + b))]

array([2, 3, 3, 4, 1])

In [ ]:
def check_avg(x):
  import statistics
  a, b, c = x["Qualitative"], x["Quantitative"], x["Experiment"]
  no_zeros = np.array(a + b + c)[np.nonzero(np.array(a + b + c))]
  avg = statistics.mean(no_zeros)
  sa, sb, sc = sum(a), sum(b), sum(c) 
  savg = statistics.mean(np.array([sa, sb, sc])[np.nonzero(np.array([sa , sb , sc]))])  # Avg of non zero sum of scores in each category 
  if (sum(a) > sum(b)) and (sum(a) > sum(c)):  # Majority: Qualitative
    if (sb > savg) or (sc > savg):
      return "Mixed Category"
    for i in b:
      if i > avg or i > 5 :   #
        return "Mixed Category"
    for j in c:
      if j > avg or i > 5  :   #or i > 0.01
        return "Mixed Category"
      
  elif (sum(b) > sum(a)) and (sum(b) > sum(c)):  # Majority: Quantitative
    if (sa > savg) or (sc > savg):
      return "Mixed Category"
    for i in a:
      if i > avg or i > 5 :   #or i > 0.01
        return "Mixed Category"
    for j in c:
      if j > avg or i > 5 :   #or i > 0.01
        return "Mixed Category"

  else:                                       # Majority : Experiment
    if (sa > savg) or (sb > savg):
      return "Mixed Category"  
    for i in a:
      if i > avg or i > 5 :
        return "Mixed Category"
    for j in b:
      if j > avg or i > 5  :   #or i > 0.01
        return "Mixed Category" 




In [ ]:
df_merged_q["Mixed_Category"] = df_merged_q[["Qualitative", "Quantitative", "Experiment"]].apply(check_avg, axis = 1)

In [ ]:
df_merged_q["Majority_Method"] = df_merged_q.apply(lambda x: "Mixed_Methods" if x["Mixed_Category"] == "Mixed Category" else x["Majority_Method"], axis = 1)

In [ ]:
df_merged_others["Majority_Method"] = df_merged_others.dropna().apply(lambda x: "Mixed_Methods" if "," in x["Majority_Method"] else x["Majority_Method"], axis = 1 )

In [ ]:
df_all = pd.concat([df_merged_q, df_merged_others])

In [ ]:
df_all["Majority_Method"].value_counts()

Quantitative     99
Qualitative      53
Mixed_Methods    53
Experiment        7
Simulation        6
Name: Majority_Method, dtype: int64

In [ ]:
df_all.head(15)

,DOI,Qualitative,Simulation,Experiment,Quantitative,Majority_Method,Mixed_Category
0,10.1177_0001839213486759,"[0, 0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[7, 0, 1, 0, 2, 3, 3, 1, 6, 0, 0]",Quantitative,None
1,10.1177_0001839218793128,"[0, 2, 0, 0, 0]","[1, 0]","[0, 0, 0, 0]","[4, 0, 2, 0, 1, 2, 3, 0, 2, 0, 0]",Quantitative,None
2,10.1177_0001839212437519,"[4, 8, 0, 0, 0]","[0, 0]","[0, 0, 1, 2]","[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Qualitative,None
3,10.1177_0001839216673823,"[0, 0, 0, 0, 0]","[0, 0]","[0, 0, 0, 0]","[6, 0, 0, 0, 3, 5, 7, 0, 3, 0, 0]",Quantitative,None
4,10.1177_0001839213488773,"[0, 0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[4, 0, 0, 0, 0, 2, 3, 0, 5, 0, 1]",Quantitative,None
5,10.1177_0001839218761369,"[0, 0, 0, 0, 0]","[1, 1]","[0, 0, 0, 0]","[0, 0, 1, 0, 2, 3, 4, 8, 6, 0, 1]",Quantitative,None
6,10.1177_0001839215619198,"[0, 3, 0, 5, 0]","[0, 0]","[0, 0, 0, 0]","[10, 0, 1, 0, 1, 7, 5, 1, 1, 0, 0]",Mixed_Methods,Mixed Category
7,10.1177_0001839219855329,"[0, 2, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Qualitative,None
8,10.1177_0001839212461439,"[0, 0, 0, 0, 0]","[0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0, 0, 11, 3, 0, 0, 0, 4]",Quantitative,None
9,10.1177_0001839212440972,"[0, 0, 0, 0, 0]","[0, 0]","[0, 0, 43, 7]","[3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",Experiment,None


In [ ]:
df_all.drop("Mixed_Category", axis = 1, inplace = True)

In [ ]:
df_all

,DOI,Qualitative,Simulation,Experiment,Quantitative,Majority_Method
0,10.1177_0001839213486759,"[0, 0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[7, 0, 1, 0, 2, 3, 3, 1, 6, 0, 0]",Quantitative
1,10.1177_0001839218793128,"[0, 2, 0, 0, 0]","[1, 0]","[0, 0, 0, 0]","[4, 0, 2, 0, 1, 2, 3, 0, 2, 0, 0]",Quantitative
2,10.1177_0001839212437519,"[4, 8, 0, 0, 0]","[0, 0]","[0, 0, 1, 2]","[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Qualitative
3,10.1177_0001839216673823,"[0, 0, 0, 0, 0]","[0, 0]","[0, 0, 0, 0]","[6, 0, 0, 0, 3, 5, 7, 0, 3, 0, 0]",Quantitative
4,10.1177_0001839213488773,"[0, 0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[4, 0, 0, 0, 0, 2, 3, 0, 5, 0, 1]",Quantitative
...,...,...,...,...,...,...
145,10.1177_0001839212439994,"[0, 0, 0, 0, 0]","[0, 0]","[0, 0, 2, 1]","[2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",Mixed_Methods
173,10.1177_0001839214538021,"[0, 0, 0, 0, 0]","[10, 0]","[0, 0, 1, 1]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",Simulation
217,asqu.2011.56.1.095,"[0, 0, 0, 1, 0]","[1, 0]","[0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Mixed_Methods
218,10.1177_0001839217726545,"[0, 0, 0, 0, 0]","[0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",NaN


In [ ]:
df_all.rename(columns = {"Qualitative" : "Qualitative [interview , interviews, ethnograph, ethnographic, qualitative, grounded theory]", "Simulation" : "Simulation [simulation, formal model]", \
                    "Experiment" : "Experiment [field experiment, lab experiment, experiment, experiments]", "Quantitative" : "Quantitative [regression, quantitative, statistically significant, statistical significance, independent_variable, control_variable, dependent_variable, logit, standard error, OLS, linear model]" }, inplace = True)

In [ ]:
df_all.to_excel(path + "Analysis/Methods_wordcount_new_2.xlsx")

**Merge with Attributes**

In [ ]:
m = pd.read_excel(path + "Analysis/Methods_wordcount_new_2.xlsx")

In [ ]:
m.drop("Unnamed: 0", axis = 1, inplace = True)

In [ ]:
#m.rename(columns = {"Qualitative" : "Qualitative [interview , ethnograph, qualitative, grounded theory]", "Simulation" : "Simulation [simulation, formal model]", \
 #                   "Experiment" : "Experiment [field experiment, lab experiment]", "Quantitative" : "Quantitative [regression, hypothesis]" }, inplace = True)


In [ ]:
m.head()


,DOI,"Qualitative [interview , interviews, ethnograph, ethnographic, qualitative, grounded theory]","Simulation [simulation, formal model]","Experiment [field experiment, lab experiment, experiment, experiments]","Quantitative [regression, quantitative, statistically significant, statistical significance, independent_variable, control_variable, dependent_variable]",Majority_Method
0,10.1177_0001839213486759,"[0, 0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[7, 0, 1, 0, 2, 3, 3, 1, 6, 0, 0]",Quantitative
1,10.1177_0001839218793128,"[0, 2, 0, 0, 0]","[1, 0]","[0, 0, 0, 0]","[4, 0, 2, 0, 1, 2, 3, 0, 2, 0, 0]",Quantitative
2,10.1177_0001839212437519,"[4, 8, 0, 0, 0]","[0, 0]","[0, 0, 1, 2]","[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Qualitative
3,10.1177_0001839216673823,"[0, 0, 0, 0, 0]","[0, 0]","[0, 0, 0, 0]","[6, 0, 0, 0, 3, 5, 7, 0, 3, 0, 0]",Quantitative
4,10.1177_0001839213488773,"[0, 0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[4, 0, 0, 0, 0, 2, 3, 0, 5, 0, 1]",Quantitative


In [ ]:
m["DOI"] = m["DOI"].apply(lambda x: x.replace("_", "/"))

In [ ]:
att = pd.read_excel(path + "Final_Topics/ASQ_Article Attributes 2011-20.xlsx")

In [ ]:
att.columns

Index(['DOI', 'Authors', 'Gender 1st A', 'Gender Other', 'Affiliation', 'US',
       'Canada', 'Europe', 'Asia', 'Other', 'Article-title', 'Subtitle',
       'Publication Date', 'Year', 'Volume', 'Issue', 'Total Citations',
       'Average per Year', 'Method', 'Data Source', 'Analysis Level',
       'Abstract', 'Unnamed: 22'],
      dtype='object')

In [ ]:
methods_merged = pd.merge(m, att[["DOI", "Year", "Article-title", "Abstract"]], on = "DOI", how = "outer")

In [ ]:
methods_merged.head()

,DOI,"Qualitative [interview , interviews, ethnograph, ethnographic, qualitative, grounded theory]","Simulation [simulation, formal model]","Experiment [field experiment, lab experiment, experiment, experiments]","Quantitative [regression, quantitative, statistically significant, statistical significance, independent_variable, control_variable, dependent_variable]",Majority_Method,Year,Article-title,Abstract
0,10.1177/0001839213486759,"[0, 0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[7, 0, 1, 0, 2, 3, 3, 1, 6, 0, 0]",Quantitative,2013,Specialization and Career Dynamics,"In this article, we attempt to resolve the ten..."
1,10.1177/0001839218793128,"[0, 2, 0, 0, 0]","[1, 0]","[0, 0, 0, 0]","[4, 0, 2, 0, 1, 2, 3, 0, 2, 0, 0]",Quantitative,2019,Dispositional Sources of Managerial Discretion...,We investigate the dispositional sources of ma...
2,10.1177/0001839212437519,"[4, 8, 0, 0, 0]","[0, 0]","[0, 0, 1, 2]","[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Qualitative,2011,Transcending Socialization,A nine-year ethnography is used to show how tw...
3,10.1177/0001839216673823,"[0, 0, 0, 0, 0]","[0, 0]","[0, 0, 0, 0]","[6, 0, 0, 0, 3, 5, 7, 0, 3, 0, 0]",Quantitative,2017,These Unequal States: Corporate Organization a...,In an analysis of data on employment in the 48...
4,10.1177/0001839213488773,"[0, 0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[4, 0, 0, 0, 0, 2, 3, 0, 5, 0, 1]",Quantitative,2013,"CEO Narcissism, Audience Engagement, and Organ...",We examine the responses of major pharmaceutic...


In [ ]:
methods_merged.to_excel(path + "Analysis/Methods/Methods_merged_wc_2.xlsx", index = False)

Word count of each keyword - for charts (comparison) - need this for the stacked bar chart 

In [ ]:
qualitative = ["interview" , "ethnograph", "qualitative", "grounded theory"]
simulation = ["simulation", "formal model"]
#experiment = ["field experiment", "lab experiment"]
experiment = ["field experiment", "lab experiment", "experiment"]
#quantitative = ["regression", "hypothesis"]
quantitative = ["regression", "quantitative", "statistically significant", "statistical significance"]
from collections import defaultdict
countwords = {}
for i in range(len(docs)):

  interview = docs[i].lower().count("interview") 
  ethnograph = docs[i].lower().count("ethnograph")
  qualitative = docs[i].lower().count("qualitative")
  grounded = docs[i].lower().count("grounded theory")
  simulation = docs[i].lower().count("simulation")
  formal = docs[i].lower().count("formal model")
  field = docs[i].lower().count("field experiment")
  lab = docs[i].lower().count("lab experiment")
  exp = docs[i].lower().count("experiment")
  regression = docs[i].lower().count("regression")
  quantitative = docs[i].lower().count("quantitative")
  ss1 = docs[i].lower().count("statistically significant")
  ss2 = docs[i].lower().count("statistical significance")


  countwords[allfiles[i].split("/")[-1].strip('.txt')] = [interview, ethnograph, qualitative, grounded, simulation, formal, field, lab, exp, regression, quantitative, ss1, ss2]

In [ ]:
df_words = pd.DataFrame(data = countwords, index = None).T

In [ ]:
df_words.reset_index(inplace = True)

In [ ]:
df_words.head()

,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,10.1177_0001839213486759,1,0,1,0,0,0,0,0,0,7,0,1,0
1,10.1177_0001839218793128,4,0,0,0,1,0,0,0,7,4,0,2,0
2,10.1177_0001839212437519,20,5,0,1,0,0,0,0,5,0,2,0,0
3,10.1177_0001839216673823,0,0,1,0,0,0,0,0,0,6,0,0,0
4,10.1177_0001839213488773,2,0,1,0,0,0,0,0,2,4,0,0,0


In [ ]:
df_words.columns = ["DOI", "interview" , "ethnograph", "qualitative", "grounded theory", "simulation", "formal model", "field experiment", "lab experiment", "experiment", "regression", "quantitative", "statistically significant", "statistical significance" ]

In [ ]:
df_words["DOI_new"] = df_words["DOI"].apply(lambda x: x.replace("_", "/"))

In [ ]:
df_words.drop("DOI", axis = 1, inplace = True)

In [ ]:
df_words.rename(columns = {"DOI_new" : "DOI"}, inplace = True)

Merge with attributes (for charts contd)

In [ ]:
att = pd.read_excel(path + "Final_Topics/ASQ_Article Attributes 2011-20.xlsx")

In [ ]:
att.columns

Index(['DOI', 'Authors', 'Gender 1st A', 'Gender Other', 'Affiliation', 'US',
       'Canada', 'Europe', 'Asia', 'Other', 'Article-title', 'Subtitle',
       'Publication Date', 'Year', 'Volume', 'Issue', 'Total Citations',
       'Average per Year', 'Method', 'Data Source', 'Analysis Level',
       'Abstract', 'Unnamed: 22'],
      dtype='object')

In [ ]:
methods_merged = pd.merge(df_words, att[["DOI", "Year", "Article-title"]], on = "DOI", how = "outer")

In [ ]:
methods_merged.head()

,interview,ethnograph,qualitative,grounded theory,simulation,formal model,field experiment,lab experiment,experiment,regression,quantitative,statistically significant,statistical significance,DOI,Year,Article-title
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,0.0,10.1177/0001839213486759,2013,Specialization and Career Dynamics
1,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0,4.0,0.0,2.0,0.0,10.1177/0001839218793128,2019,Dispositional Sources of Managerial Discretion...
2,20.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,10.1177/0001839212437519,2011,Transcending Socialization
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,10.1177/0001839216673823,2017,These Unequal States: Corporate Organization a...
4,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,10.1177/0001839213488773,2013,"CEO Narcissism, Audience Engagement, and Organ..."


In [ ]:
methods_merged.to_excel(path + "Analysis/methods_words.xlsx")

**Mixed Category - Qualitative and Quantitative**

Decision Rule: Above average OR Above a number (10)

In [ ]:
df_merged = pd.merge(df_methods[["DOI", "Majority_Method"]], df_countwords, on = "DOI", how = "outer")

In [ ]:
df_merged["Majority_Method"].value_counts()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[Qualitative]                  107
[Quantitative]                  83
[Experiment]                     9
[Simulation]                     7
[Experiment, Quantitative]       5
[Simulation, Quantitative]       3
[Qualitative, Quantitative]      2
[Qualitative, Simulation]        2
Name: Majority_Method, dtype: int64

In [ ]:
df_merged

,DOI,Majority_Method,"[interview , ethnograph, qualitative, grounded theory]","[simulation, formal model]","[field experiment, lab experiment, experiment]","[regression, quantitative, statistically significant, statistical significance]"
0,10.1177_0001839213486759,[Quantitative],"[1, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[7, 0, 1, 0]"
1,10.1177_0001839218793128,[Quantitative],"[4, 0, 0, 0]","[1, 0]","[0, 0, 0, 0]","[4, 0, 2, 0]"
2,10.1177_0001839212437519,[Qualitative],"[20, 5, 0, 1]","[0, 0]","[0, 0, 1, 2]","[0, 2, 0, 0]"
3,10.1177_0001839216673823,[Quantitative],"[0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[6, 0, 0, 0]"
4,10.1177_0001839213488773,[Quantitative],"[2, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[4, 0, 0, 0]"
...,...,...,...,...,...,...
215,asqu.2011.56.1.061,[Quantitative],"[3, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[8, 1, 0, 0]"
216,asqu.2011.56.1.026,[Quantitative],"[0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[4, 0, 8, 0]"
217,asqu.2011.56.1.095,"[Qualitative, Simulation]","[0, 0, 1, 0]","[1, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]"
218,10.1177_0001839212461439,NaN,"[0, 0, 0, 0]","[0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]"


In [ ]:
df_merged["Majority_Method_cs"] = df_merged["Majority_Method"].dropna().apply(lambda x: ",".join(x))

In [ ]:
m = ["Qualitative", "Quantitative"]
df_merged_q = df_merged[(df_merged["Majority_Method_cs"] == "Qualitative") | (df_merged["Majority_Method_cs"] == "Quantitative") ].drop("Majority_Method", axis =1)
df_merged_q.columns = ["DOI", 'Qualitative', 'Simulation', 'Experiment', 'Quantitative', 'Majority_Method' ]

In [ ]:
df_merged_others = df_merged[(df_merged["Majority_Method_cs"] != "Qualitative") & (df_merged["Majority_Method_cs"] != "Quantitative") ].drop("Majority_Method", axis =1)
df_merged_others.columns = ["DOI", 'Qualitative', 'Simulation', 'Experiment', 'Quantitative', 'Majority_Method' ]

In [ ]:
import statistics
a = [0,2,3]
b = [3,4,1]
statistics.mean(a + b)
np.array(a + b)[np.nonzero(np.array(a + b))]

array([2, 3, 3, 4, 1])

In [ ]:
def check_avg(x):
  import statistics
  a, b = x["Qualitative"], x["Quantitative"]
  no_zeros = np.array(a + b)[np.nonzero(np.array(a + b))]
  avg = statistics.mean(no_zeros)
  if sum(a) > sum(b):
    for i in b:
      if i > avg or i > 5:
        return "Mixed Category"
      
  else:
    for i in a:
      if i > avg or i > 5:
        return "Mixed Category"




In [ ]:
df_merged_q["Mixed_Category"] = df_merged_q[["Qualitative", "Quantitative"]].apply(check_avg, axis = 1)

In [ ]:
df_merged_q["Majority_Method"] = df_merged_q.apply(lambda x: "Mixed_Methods" if x["Mixed_Category"] == "Mixed Category" else x["Majority_Method"], axis = 1)

In [ ]:
df_merged_others["Majority_Method"] = df_merged_others.dropna().apply(lambda x: "Mixed_Methods" if "," in x["Majority_Method"] else x["Majority_Method"], axis = 1 )

In [ ]:
df_all = pd.concat([df_merged_q, df_merged_others])

In [ ]:
df_all

,DOI,Qualitative,Simulation,Experiment,Quantitative,Majority_Method,Mixed_Category
0,10.1177_0001839213486759,"[1, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[7, 0, 1, 0]",Quantitative,None
1,10.1177_0001839218793128,"[4, 0, 0, 0]","[1, 0]","[0, 0, 0, 0]","[4, 0, 2, 0]",Mixed_Methods,Mixed Category
2,10.1177_0001839212437519,"[20, 5, 0, 1]","[0, 0]","[0, 0, 1, 2]","[0, 2, 0, 0]",Qualitative,None
3,10.1177_0001839216673823,"[0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[6, 0, 0, 0]",Quantitative,None
4,10.1177_0001839213488773,"[2, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[4, 0, 0, 0]",Quantitative,None
...,...,...,...,...,...,...,...
201,10.1177_0001839214523428,"[0, 0, 0, 0]","[4, 0]","[0, 0, 0, 1]","[0, 0, 1, 1]",Simulation,NaN
214,asqu.2011.56.1.001,"[1, 0, 0, 0]","[0, 0]","[0, 0, 30, 10]","[4, 1, 0, 0]",Experiment,NaN
217,asqu.2011.56.1.095,"[0, 0, 1, 0]","[1, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]",Mixed_Methods,NaN
218,10.1177_0001839212461439,"[0, 0, 0, 0]","[0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]",NaN,NaN


In [ ]:
df_all["Majority_Method"].value_counts()

Qualitative      90
Quantitative     64
Mixed_Methods    48
Experiment        9
Simulation        7
Name: Majority_Method, dtype: int64

In [ ]:
df_all.drop("Mixed_Category", axis = 1, inplace = True)

In [ ]:
df_all

,DOI,Qualitative,Simulation,Experiment,Quantitative,Majority_Method
0,10.1177_0001839213486759,"[1, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[7, 0, 1, 0]",Quantitative
1,10.1177_0001839218793128,"[4, 0, 0, 0]","[1, 0]","[0, 0, 0, 0]","[4, 0, 2, 0]",Mixed_Methods
2,10.1177_0001839212437519,"[20, 5, 0, 1]","[0, 0]","[0, 0, 1, 2]","[0, 2, 0, 0]",Qualitative
3,10.1177_0001839216673823,"[0, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[6, 0, 0, 0]",Quantitative
4,10.1177_0001839213488773,"[2, 0, 1, 0]","[0, 0]","[0, 0, 0, 0]","[4, 0, 0, 0]",Quantitative
...,...,...,...,...,...,...
201,10.1177_0001839214523428,"[0, 0, 0, 0]","[4, 0]","[0, 0, 0, 1]","[0, 0, 1, 1]",Simulation
214,asqu.2011.56.1.001,"[1, 0, 0, 0]","[0, 0]","[0, 0, 30, 10]","[4, 1, 0, 0]",Experiment
217,asqu.2011.56.1.095,"[0, 0, 1, 0]","[1, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]",Mixed_Methods
218,10.1177_0001839212461439,"[0, 0, 0, 0]","[0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]",NaN


In [ ]:
df_all.rename(columns = {"Qualitative" : "Qualitative [interview , ethnograph, qualitative, grounded theory]", "Simulation" : "Simulation [simulation, formal model]", \
                    "Experiment" : "Experiment [field experiment, lab experiment, experiment, experiments]", "Quantitative" : "Quantitative [regression, quantitative, statistically significant, statistical significance]" }, inplace = True)

In [ ]:
df_all.to_excel(path + "Analysis/Methods_new.xlsx")

**Methods -Merged with attributes**

In [ ]:
m = pd.read_excel(path + "Analysis/Methods_new.xlsx")

In [ ]:
m.drop("Unnamed: 0", axis = 1, inplace = True)

In [ ]:
#m.rename(columns = {"Qualitative" : "Qualitative [interview , ethnograph, qualitative, grounded theory]", "Simulation" : "Simulation [simulation, formal model]", \
 #                   "Experiment" : "Experiment [field experiment, lab experiment]", "Quantitative" : "Quantitative [regression, hypothesis]" }, inplace = True)


In [ ]:
m.head()

,DOI,"Qualitative [interview , ethnograph, qualitative, grounded theory]","Simulation [simulation, formal model]","Experiment [field experiment, lab experiment, experiment]","Quantitative [regression, quantitative, statistically significant, statistical significance]",Majority_Method
0,10.1177_0001839213486759,"[1, 0, 1, 0]","[0, 0]","[0, 0, 0]","[7, 0, 1, 0]",Quantitative
1,10.1177_0001839212437519,"[20, 5, 0, 1]","[0, 0]","[0, 0, 5]","[0, 2, 0, 0]",Qualitative
2,10.1177_0001839216673823,"[0, 0, 1, 0]","[0, 0]","[0, 0, 0]","[6, 0, 0, 0]",Mixed_Methods
3,10.1177_0001839213488773,"[2, 0, 1, 0]","[0, 0]","[0, 0, 2]","[4, 0, 0, 0]",Mixed_Methods
4,10.1177_0001839219855329,"[6, 13, 1, 1]","[0, 0]","[0, 0, 2]","[0, 1, 0, 0]",Qualitative


In [ ]:
m["DOI_new"] = m["DOI"].apply(lambda x: x.replace("_", "/"))

In [ ]:
m.drop("DOI", axis = 1, inplace = True)

In [ ]:
m.rename(columns = {"DOI_new" : "DOI"}, inplace = True)

In [ ]:
att.columns

Index(['DOI', 'Authors', 'Gender 1st A', 'Gender Other', 'Affiliation', 'US',
       'Canada', 'Europe', 'Asia', 'Other', 'Article-title', 'Subtitle',
       'Publication Date', 'Year', 'Volume', 'Issue', 'Total Citations',
       'Average per Year', 'Method', 'Data Source', 'Analysis Level',
       'Abstract', 'Unnamed: 22'],
      dtype='object')

In [ ]:
methods_merged = pd.merge(m, att[["DOI", "Year", "Article-title", "Abstract"]], on = "DOI", how = "outer")

In [ ]:
methods_merged.head()

,"Qualitative [interview , ethnograph, qualitative, grounded theory]","Simulation [simulation, formal model]","Experiment [field experiment, lab experiment, experiment]","Quantitative [regression, quantitative, statistically significant, statistical significance]",Majority_Method,DOI,Year,Article-title,Abstract
0,"[1, 0, 1, 0]","[0, 0]","[0, 0, 0]","[7, 0, 1, 0]",Quantitative,10.1177/0001839213486759,2013,Specialization and Career Dynamics,"In this article, we attempt to resolve the ten..."
1,"[20, 5, 0, 1]","[0, 0]","[0, 0, 5]","[0, 2, 0, 0]",Qualitative,10.1177/0001839212437519,2011,Transcending Socialization,A nine-year ethnography is used to show how tw...
2,"[0, 0, 1, 0]","[0, 0]","[0, 0, 0]","[6, 0, 0, 0]",Mixed_Methods,10.1177/0001839216673823,2017,These Unequal States: Corporate Organization a...,In an analysis of data on employment in the 48...
3,"[2, 0, 1, 0]","[0, 0]","[0, 0, 2]","[4, 0, 0, 0]",Mixed_Methods,10.1177/0001839213488773,2013,"CEO Narcissism, Audience Engagement, and Organ...",We examine the responses of major pharmaceutic...
4,"[6, 13, 1, 1]","[0, 0]","[0, 0, 2]","[0, 1, 0, 0]",Qualitative,10.1177/0001839219855329,2020,Evaluative Spillovers from Technological Chang...,Most studies of technologies’ impact on occupa...


In [ ]:
methods_merged.to_excel(path + "Analysis/Methods_merged_atts.xlsx", index = False)

**Spreadsheet for stacked bar chart (methods)**

In [ ]:
m = pd.read_excel(path + "Analysis/methods_words.xlsx")

In [ ]:
m.head()

,Unnamed: 0,interview,ethnograph,qualitative,grounded theory,simulation,formal model,field experiment,lab experiment,experiment,regression,quantitative,statistically significant,statistical significance,DOI,Year,Article-title
0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,0.0,10.1177/0001839213486759,2013,Specialization and Career Dynamics
1,1,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0,4.0,0.0,2.0,0.0,10.1177/0001839218793128,2019,Dispositional Sources of Managerial Discretion...
2,2,20.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,10.1177/0001839212437519,2011,Transcending Socialization
3,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,10.1177/0001839216673823,2017,These Unequal States: Corporate Organization a...
4,4,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,10.1177/0001839213488773,2013,"CEO Narcissism, Audience Engagement, and Organ..."


In [ ]:
m.columns

Index(['Unnamed: 0', 'interview', 'ethnograph', 'qualitative',
       'grounded theory', 'simulation', 'formal model', 'field experiment',
       'lab experiment', 'experiment', 'regression', 'quantitative',
       'statistically significant', 'statistical significance', 'DOI', 'Year',
       'Article-title'],
      dtype='object')

In [ ]:
methods_grouped = m.groupby("Year").agg(lambda x: x.ne(0).sum())

In [ ]:
methods_grouped.head()

,Unnamed: 0,interview,ethnograph,qualitative,grounded theory,simulation,formal model,field experiment,lab experiment,experiment,regression,quantitative,statistically significant,statistical significance,DOI,Article-title
Year,,,,,,,,,,,,,,,,
2011,17,9.0,2.0,6.0,1.0,2.0,1.0,1.0,1.0,15.0,12.0,6.0,3.0,1.0,17,17
2012,16,8.0,2.0,8.0,1.0,4.0,0.0,1.0,0.0,12.0,11.0,9.0,7.0,3.0,16,16
2013,16,9.0,3.0,11.0,1.0,1.0,0.0,1.0,0.0,8.0,15.0,1.0,8.0,2.0,17,17
2014,22,14.0,4.0,14.0,2.0,5.0,2.0,3.0,2.0,15.0,14.0,10.0,11.0,6.0,22,22
2015,21,16.0,3.0,11.0,4.0,2.0,1.0,3.0,1.0,9.0,14.0,8.0,8.0,2.0,21,21


In [ ]:
methods_grouped.drop("Unnamed: 0", axis = 1, inplace = True)

In [ ]:
methods_grouped.drop(["DOI", "Article-title"], axis = 1, inplace = True)

In [ ]:
methods_grouped.reset_index(inplace = True)

In [ ]:
methods_grouped.to_excel(path + "Analysis/methods_grouped_yearwise.xlsx")

In [ ]:
methods_narrow = methods_grouped.melt(id_vars=['Year'])

In [ ]:
import pandas
from openpyxl import load_workbook

book = load_workbook(path + "Analysis/methods_grouped_yearwise.xlsx")
writer = pandas.ExcelWriter(path + "Analysis/methods_grouped_yearwise.xlsx", engine='openpyxl') 
writer.book = book

In [ ]:
methods_narrow.to_excel(writer, sheet_name='Sheet2')

writer.save()

In [ ]:
#methods_narrow.to_excel(path + "Analysis/methods_grouped_yearwise.xlsx", sheet_name='Sheet2')

**QCA**

In [ ]:
qca = ["qca", "qualitative comparative analysis"]


In [ ]:
qca = ["qca", "qualitative comparative analysis"]
from collections import defaultdict
countqca = {}
for i in range(len(docs)):

  #qca = docs[i].lower().count("qca") 
  #quali = docs[i].lower().count("qualitative comparative analysis")
  quali = [docs[i].lower().count(x) for x in qca]


  countqca[allfiles[i].split("/")[-1].strip('.txt')] = quali

In [ ]:
qca = pd.DataFrame(data = countqca, index = None).T

In [ ]:
qca.reset_index(inplace = True)

In [ ]:
qca.head()

,index,0,1
0,10.1177_0001839213486759,0,0
1,10.1177_0001839218793128,0,0
2,10.1177_0001839212437519,0,0
3,10.1177_0001839216673823,0,0
4,10.1177_0001839213488773,0,0


In [ ]:
qca.columns = ["DOI", "qca", "qualitative comparative analysis"] 

In [ ]:
qca["DOI_new"] = qca["DOI"].apply(lambda x: x.replace("_", "/"))

In [ ]:
qca.drop("DOI", axis = 1, inplace = True)

In [ ]:
qca.rename(columns = {"DOI_new" : "DOI"}, inplace = True)

In [ ]:
qca["qca"].value_counts()

0    218
3      1
1      1
Name: qca, dtype: int64

In [ ]:
qca["qualitative comparative analysis"].value_counts()

0    219
1      1
Name: qualitative comparative analysis, dtype: int64

merge with attributes

In [ ]:
qca_merged = pd.merge(att[["DOI", "Article-title", "Abstract"]], qca, on = "DOI", how = "outer")

In [ ]:
qca_merged

,DOI,Article-title,Abstract,qca,qualitative comparative analysis
0,10.1177/0001839217725782,The Artisan and His Audience: Identification w...,"Using ethnographic, experimental, and survey d...",0.0,0.0
1,10.1177/0001839215591627,Business as Plan B,This article develops and empirically evaluate...,0.0,0.0
2,10.1177/0001839218763595,How Misconduct Spreads: Auditors‚Äô Role in th...,I study the role of external auditors in the d...,0.0,0.0
3,10.1177/0001839218793128,Dispositional Sources of Managerial Discretion...,We investigate the dispositional sources of ma...,0.0,0.0
4,10.1177/0001839218761369,Network Synergy,Acquisitions can dramatically reshape interorg...,0.0,0.0
...,...,...,...,...,...
225,10.1177/0001839220963633,Escaping the Ellipsis of Diversity: Insider Ac...,Researchers have explored in depth how social ...,0.0,0.0
226,asqu.2011.56.1.001,The Ethical Dangers of Deliberative Decision M...,Research on ethical decision making has been h...,0.0,0.0
227,asqu.2011.56.1.026,When Firms Are Desperate to Grow via Acquisiti...,In this paper we draw on work in behavioral le...,0.0,0.0
228,asqu.2011.56.1.061,Identities as Lenses: How Organizational Ident...,This study calls into question the completenes...,0.0,0.0


In [ ]:
qca_merged.to_excel(path +"Analysis/qca.xlsx")